In [ ]:
import torch
import pandas as pd
import io, sys, os
from functools import reduce
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
import random
import re
nlp = English()
tokenizer = nlp.tokenizer
from tqdm.notebook import tqdm
import plotly.express as px

from sentence_transformers import SentenceTransformer
from datasets import load_metric
from torch import nn
from scipy import stats
from scipy.spatial.distance import euclidean

import matplotlib.pyplot as plt

In [ ]:
# source_lang = "si" ## ru, si, et, ro, ne (Russian, Sinhala, Estonian, Romanian, Nepali)
# tqmodel = "unsupervised" ## DO NOT CHANGE THIS AS THIS NOTEBOOK ONLY SUPPORTS THE UNSUPERVISED METHOD

In [ ]:
df_trans_thres_et = []
dfdevet = pd.read_csv("et_en_data/dev.eten.df.short.tsv",sep='\t')
dftestet = pd.read_csv("et_en_data/test20.eten.df.short.tsv", sep='\t')
frameset = [dfdevet, dftestet]
finalet = pd.concat(frameset)
combet = finalet.reset_index(drop=True)
devtestCombet = combet.drop(columns=['index', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
df_trans_thres_ru = []
dfdevru = pd.read_csv("ru_en_data/dev.ruen.df.short.tsv",sep='\t')
dftestru = pd.read_csv("ru_en_data/test20.ruen.df.short.tsv", sep='\t')
framesru = [dfdevru, dftestru]
finalru = pd.concat(framesru)
combru = finalru.reset_index(drop=True)
devtestCombru = combru.drop(columns=['segid', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
df_trans_thres_ro = []
dfdevro = pd.read_csv("ro_en_data/dev.roen.df.short.tsv",sep='\t')
dftestro = pd.read_csv("ro_en_data/test20.roen.df.short.tsv", sep='\t')
framesro = [dfdevro, dftestro]
finalro = pd.concat(framesro)
combro = finalro.reset_index(drop=True)
devtestCombro = combro.drop(columns=['index', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
df_trans_thres_si = []
dfdevsi = pd.read_csv("si_en_data/dev.sien.df.short.tsv",sep='\t')
dftestsi = pd.read_csv("si_en_data/test20.sien.df.short.tsv", sep='\t')
framessi = [dfdevsi, dftestsi]
finalsi = pd.concat(framessi)
combsi = finalsi.reset_index(drop=True)
devtestCombsi = combsi.drop(columns=['index', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
df_trans_thres_ne = []
dfdevne = pd.read_csv("ne_en_data/dev.neen.df.short.tsv",sep='\t')
dftestne = pd.read_csv("ne_en_data/test20.neen.df.short.tsv", sep='\t')
framesne = [dfdevne, dftestne]
finalne = pd.concat(framesne)
combne = finalne.reset_index(drop=True)
devtestCombne = combne.drop(columns=['index', 'mean', 'z_scores', 'z_mean', 'model_scores', 'scores'])

In [ ]:
from transquest.algo.sentence_level.siamesetransquest.run_model import SiameseTransQuestModel
modelEt = SiameseTransQuestModel("TransQuest/siamesetransquest-da-et_en-wiki")
modelRu = SiameseTransQuestModel("TransQuest/siamesetransquest-da-ru_en-reddit_wikiquotes")
modelRo = SiameseTransQuestModel("TransQuest/siamesetransquest-da-ro_en-wiki")
modelSi = SiameseTransQuestModel("TransQuest/siamesetransquest-da-si_en-wiki")
modelNe = SiameseTransQuestModel("TransQuest/siamesetransquest-da-ne_en-wiki")

In [ ]:
dtlolet = devtestCombet.values.tolist()
nppredictionset = modelEt.predict(dtlolet) ## For Siamese
df_orig_trans_scoreet = devtestCombet
df_orig_trans_scoreet['trans_score_siam'] = nppredictionset

In [ ]:
dtlolru = devtestCombru.values.tolist()
nppredictionsru = modelRu.predict(dtlolru) ## For Siamese
df_orig_trans_scoreru = devtestCombru
df_orig_trans_scoreru['trans_score_siam'] = nppredictionsru

In [ ]:
dtlolro = devtestCombro.values.tolist()
nppredictionsro = modelRo.predict(dtlolro) ## For Siamese
df_orig_trans_scorero = devtestCombro
df_orig_trans_scorero['trans_score_siam'] = nppredictionsro

In [ ]:
dtlolsi = devtestCombsi.values.tolist()
nppredictionssi = modelSi.predict(dtlolsi) ## For Siamese
df_orig_trans_scoresi = devtestCombsi
df_orig_trans_scoresi['trans_score_siam'] = nppredictionssi

In [ ]:
dtlolne = devtestCombne.values.tolist()
nppredictionsne = modelNe.predict(dtlolne) ## For Siamese
df_orig_trans_scorene = devtestCombne
df_orig_trans_scorene['trans_score_siam'] = nppredictionsne

In [ ]:
l = combet['scores'].tolist()
newlist = []
sublist =[]
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_scoreet['trans_score_hum'] = newlistmeans

pearsonSiameseEt = stats.pearsonr(df_orig_trans_scoreet['trans_score_siam'].values.tolist(), df_orig_trans_scoreet['trans_score_hum'].values.tolist())

In [ ]:
l = combru['scores'].tolist()
newlist = []
sublist =[]
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_scoreru['trans_score_hum'] = newlistmeans

pearsonSiameseRu = stats.pearsonr(df_orig_trans_scoreru['trans_score_siam'].values.tolist(), df_orig_trans_scoreru['trans_score_hum'].values.tolist())

In [ ]:
l = combro['scores'].tolist()
newlist = []
sublist =[]
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_scorero['trans_score_hum'] = newlistmeans

pearsonSiameseRo = stats.pearsonr(df_orig_trans_scorero['trans_score_siam'].values.tolist(), df_orig_trans_scorero['trans_score_hum'].values.tolist())

In [ ]:
l = combsi['scores'].tolist()
newlist = []
sublist =[]
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_scoresi['trans_score_hum'] = newlistmeans

pearsonSiameseSi = stats.pearsonr(df_orig_trans_scoresi['trans_score_siam'].values.tolist(), df_orig_trans_scoresi['trans_score_hum'].values.tolist())

In [ ]:
l = combne['scores'].tolist()
newlist = []
sublist =[]
for sublist in l:
    sublist = sublist.split(", ")
    sublist = [i.strip('[]') for i in sublist]
    sublist = [float(i) for i in sublist]
    newlist.append(sublist)

newlistnparray = np.array(newlist)
newlistmeans = np.mean(newlistnparray, axis=1)
newlistmeans = newlistmeans/100
newlistmeans = newlistmeans.tolist()
df_orig_trans_scorene['trans_score_hum'] = newlistmeans

pearsonSiameseNe = stats.pearsonr(df_orig_trans_scorene['trans_score_siam'].values.tolist(), df_orig_trans_scorene['trans_score_hum'].values.tolist())

In [ ]:
pearsonSiameseEt[0], pearsonSiameseRu[0], pearsonSiameseRo[0], pearsonSiameseSi[0], pearsonSiameseNe[0]

In [ ]:
## USE THIS TO CLEAR THE GPU MEMORY IN CASE YOU DO NOT HAVE ENOUGH GPU MEM FOR ALL FIVE MODELS AT A TIME
# !pip install numba
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

In [ ]:
from transquest.algo.sentence_level.monotransquest.run_model import MonoTransQuestModel

modelEt = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-et_en-wiki", num_labels=1, use_cuda=torch.cuda.is_available())
modelRu = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-ru_en-reddit_wikiquotes", num_labels=1, use_cuda=torch.cuda.is_available())
modelRo = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-ro_en-wiki", num_labels=1, use_cuda=torch.cuda.is_available())
modelSi = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-si_en-wiki", num_labels=1, use_cuda=torch.cuda.is_available())
modelNe = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-ne_en-wiki", num_labels=1, use_cuda=torch.cuda.is_available())

In [ ]:
nppredictionset, raw = modelEt.predict(dtlolet) 
df_orig_trans_scoreet = devtestCombet
df_orig_trans_scoreet['trans_score_mono'] = nppredictionset

In [ ]:
nppredictionsru, raw = modelRu.predict(dtlolru) 
df_orig_trans_scoreru = devtestCombru
df_orig_trans_scoreru['trans_score_mono'] = nppredictionsru

In [ ]:
nppredictionsro, raw = modelRo.predict(dtlolro) 
df_orig_trans_scorero = devtestCombro
df_orig_trans_scorero['trans_score_mono'] = nppredictionsro

In [ ]:
nppredictionssi, raw = modelRo.predict(dtlolsi)
df_orig_trans_scoresi = devtestCombsi
df_orig_trans_scoresi['trans_score_mono'] = nppredictionssi

In [ ]:
nppredictionsne, raw = modelNe.predict(dtlolne)
df_orig_trans_scorene = devtestCombne
df_orig_trans_scorene['trans_score_mono'] = nppredictionsne

In [ ]:
pearsonMonoEt = stats.pearsonr(df_orig_trans_scoreet['trans_score_mono'].values.tolist(), df_orig_trans_scoreet['trans_score_hum'].values.tolist())
pearsonMonoRu = stats.pearsonr(df_orig_trans_scoreru['trans_score_mono'].values.tolist(), df_orig_trans_scoreru['trans_score_hum'].values.tolist())
pearsonMonoRo = stats.pearsonr(df_orig_trans_scorero['trans_score_mono'].values.tolist(), df_orig_trans_scorero['trans_score_hum'].values.tolist())
pearsonMonoSi = stats.pearsonr(df_orig_trans_scoresi['trans_score_mono'].values.tolist(), df_orig_trans_scoresi['trans_score_hum'].values.tolist())
pearsonMonoNe = stats.pearsonr(df_orig_trans_scorene['trans_score_mono'].values.tolist(), df_orig_trans_scorene['trans_score_hum'].values.tolist())

In [ ]:
pearsonMonoEt[0], pearsonMonoRu[0], pearsonMonoRo[0], pearsonMonoSi[0], pearsonMonoNe[0]

In [ ]:
modelMulti = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-multilingual", num_labels=1, use_cuda=torch.cuda.is_available())

In [ ]:
nppredictionset, raw = modelMulti.predict(dtlolet) 
df_orig_trans_scoreet = devtestCombet
df_orig_trans_scoreet['trans_score_multi'] = nppredictionset

In [ ]:
nppredictionsru, raw = modelMulti.predict(dtlolru)
df_orig_trans_scoreru = devtestCombru
df_orig_trans_scoreru['trans_score_multi'] = nppredictionsru

In [ ]:
nppredictionsro, raw = modelMulti.predict(dtlolro) 
df_orig_trans_scorero = devtestCombro
df_orig_trans_scorero['trans_score_multi'] = nppredictionsro

In [ ]:
nppredictionssi, raw = modelMulti.predict(dtlolsi)
df_orig_trans_scoresi = devtestCombsi
df_orig_trans_scoresi['trans_score_multi'] = nppredictionssi

In [ ]:
nppredictionsne, raw = modelMulti.predict(dtlolne)
df_orig_trans_scorene = devtestCombne
df_orig_trans_scorene['trans_score_multi'] = nppredictionsne

In [ ]:
pearsonMultiEt = stats.pearsonr(df_orig_trans_scoreet['trans_score_multi'].values.tolist(), df_orig_trans_scoreet['trans_score_hum'].values.tolist())
pearsonMultiRu = stats.pearsonr(df_orig_trans_scoreru['trans_score_multi'].values.tolist(), df_orig_trans_scoreru['trans_score_hum'].values.tolist())
pearsonMultiRo = stats.pearsonr(df_orig_trans_scorero['trans_score_multi'].values.tolist(), df_orig_trans_scorero['trans_score_hum'].values.tolist())
pearsonMultiSi = stats.pearsonr(df_orig_trans_scoresi['trans_score_multi'].values.tolist(), df_orig_trans_scoresi['trans_score_hum'].values.tolist())
pearsonMultiNe = stats.pearsonr(df_orig_trans_scorene['trans_score_multi'].values.tolist(), df_orig_trans_scorene['trans_score_hum'].values.tolist())

In [ ]:
pearsonMultiEt[0], pearsonMultiRu[0], pearsonMultiRo[0], pearsonMultiSi[0], pearsonMultiNe[0]

In [ ]:
def combine_metrics(*args, **kwargs):
    assert len(args) == len(kwargs["corr"])
    assert len(args[0]) == len(args[1])
    
    output = []
    
    for i in range(len(args[0])):
        value = 0
        for sign, metric in zip(kwargs["corr"],args):
            if sign > 0:
                value += np.exp(metric[i])
            else:
                value += np.exp(1-metric[i])
        output.append(value)
    return output

bert_score_metric = load_metric('bertscore', keep_in_memory=False, cache_dir=sys.path[0])
bert_score_model = 'xlm-roberta-base'
cos_sim_model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

In [ ]:
def predict_scores_orig(df, nTrials=1):
    ## Need a temporary scoring function here. 
    scores = np.zeros((1, df.shape[0]))
  
    for i in tqdm(range(nTrials)):
        embed_sent1 = cos_sim_model.encode(df['translation'].values.tolist(), convert_to_tensor=True)
        embed_sent2 = cos_sim_model.encode(df['original'].values.tolist(), convert_to_tensor=True)
        cos_sim = nn.CosineSimilarity(dim=1)(embed_sent1,embed_sent2)
        cos_sim = (cos_sim -torch.min(cos_sim))/ (torch.max(cos_sim)-torch.min(cos_sim))
        cos_scores = cos_sim.cpu().numpy()

        bert_score_metric.add_batch(predictions=df['translation'].values.tolist(), references=df['original'].values.tolist())
        score = bert_score_metric.compute(model_type=bert_score_model)
        norm_score = (score["f1"] - np.min(score["f1"])) / (np.max(score["f1"]) - np.min(score["f1"]))
        compound_metric = combine_metrics(cos_scores, norm_score, corr=[1, 1])
        df_norm = pd.DataFrame()
        df_norm['scores'] = compound_metric
        df_norm['scores'] = (df_norm['scores'] - df_norm['scores'].min()) / (df_norm['scores'].max() - df_norm['scores'].min())
        predictions = df_norm['scores'].values.tolist()
    
        scores[i] = predictions

    return scores

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_orig_trans_scoreet)
df_orig_trans_scoreet['trans_score_unsupervised'] = scores_orig[0]

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_orig_trans_scoreru)
df_orig_trans_scoreru['trans_score_unsupervised'] = scores_orig[0]

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_orig_trans_scorero)
df_orig_trans_scorero['trans_score_unsupervised'] = scores_orig[0]

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_orig_trans_scoresi)
df_orig_trans_scoresi['trans_score_unsupervised'] = scores_orig[0]

In [ ]:
scores_orig=[]
scores_orig = predict_scores_orig(df_orig_trans_scorene)
df_orig_trans_scorene['trans_score_unsupervised'] = scores_orig[0]

In [ ]:
pearsonUnsupervisedEt = stats.pearsonr(df_orig_trans_scoreet['trans_score_unsupervised'].values.tolist(), df_orig_trans_scoreet['trans_score_hum'].values.tolist())
pearsonUnsupervisedRu = stats.pearsonr(df_orig_trans_scoreru['trans_score_unsupervised'].values.tolist(), df_orig_trans_scoreru['trans_score_hum'].values.tolist())
pearsonUnsupervisedRo = stats.pearsonr(df_orig_trans_scorero['trans_score_unsupervised'].values.tolist(), df_orig_trans_scorero['trans_score_hum'].values.tolist())
pearsonUnsupervisedSi = stats.pearsonr(df_orig_trans_scoresi['trans_score_unsupervised'].values.tolist(), df_orig_trans_scoresi['trans_score_hum'].values.tolist())
pearsonUnsupervisedNe = stats.pearsonr(df_orig_trans_scorene['trans_score_unsupervised'].values.tolist(), df_orig_trans_scorene['trans_score_hum'].values.tolist())

In [ ]:
pearsonUnsupervisedEt[0], pearsonUnsupervisedRu[0], pearsonUnsupervisedRo[0], pearsonUnsupervisedSi[0], pearsonUnsupervisedNe[0]

In [ ]:
## This cell has precomputed values from our experiment, the cell below shows how to recompute them from the dumped means values.

# etRanking = [-0.07716989078923159, -0.0003569826549955548, -0.04823801347576072, -0.011562646249216116, 0.009073152397363508]
# ruRanking = [-0.12114444232591343, -0.007808453126371578, -0.05595449589693772, 0.007777049013473376, -0.03316327903357645]
# roRanking = [-0.08864646480583055, -0.046775697881407216, -0.06406304652518169, -0.032233587293031205, 0.016345227061293727]
# siRanking = [-0.05806420791682877, -0.020244334000706014, -0.04098220828783415, -0.02550455062267074, -0.0550465349628457]
# neRanking = [-0.07638170781723766, -0.028229749720718452, -0.05293691062280626, -0.017175247487624883, -0.03316327921984096]

# rows = ["MonoTransQuest", "SiameseTransQuest", "MultiTransQuest", "Unsupervised", "OpenKiwi"]

# plt.figure(figsize=(9, 5))
# plt.bar(rows, etRanking)
# plt.suptitle('Et-En Ranked Systems')
# plt.show()
# plt.figure(figsize=(9, 5))
# plt.bar(rows, ruRanking)
# plt.suptitle('Ru-En Ranked Systems')
# plt.show()
# plt.figure(figsize=(9, 5))
# plt.bar(rows, siRanking)
# plt.suptitle('Si-En Ranked Systems')
# plt.show()
# plt.figure(figsize=(9, 5))
# plt.bar(rows, roRanking)
# plt.suptitle('Ro-En Ranked Systems')
# plt.show()
# plt.figure(figsize=(9, 5))
# plt.bar(rows, neRanking)
# plt.suptitle('Ne-En Ranked Systems')
# plt.show()

In [ ]:
etMeans_mono = np.load("commonResults/etMeans_mono.npy")

etMPPAvg_mono = (etMeans_mono[0] + etMeans_mono[1] + etMeans_mono[2] + etMeans_mono[3] + etMeans_mono[4] + etMeans_mono[5])/6
etMAPAvg_mono = (etMeans_mono[6] + etMeans_mono[7] + etMeans_mono[8] + etMeans_mono[9] + etMeans_mono[10] + etMeans_mono[11] + etMeans_mono[12] + etMeans_mono[13])/8

etDiff_mono = etMPPAvg_mono - etMAPAvg_mono

ruMeans_mono = np.load("commonResults/ruMeans_mono.npy")

ruMPPAvg_mono = (ruMeans_mono[0] + ruMeans_mono[1] + ruMeans_mono[2] + ruMeans_mono[3] + ruMeans_mono[4] + ruMeans_mono[5])/6
ruMAPAvg_mono = (ruMeans_mono[6] + ruMeans_mono[7] + ruMeans_mono[8] + ruMeans_mono[9] + ruMeans_mono[10] + ruMeans_mono[11] + ruMeans_mono[12] + ruMeans_mono[13])/8

ruDiff_mono = ruMPPAvg_mono - ruMAPAvg_mono

roMeans_mono = np.load("commonResults/roMeans_mono.npy")

roMPPAvg_mono = (roMeans_mono[0] + roMeans_mono[1] + roMeans_mono[2] + roMeans_mono[3] + roMeans_mono[4] + roMeans_mono[5])/6
roMAPAvg_mono = (roMeans_mono[6] + roMeans_mono[7] + roMeans_mono[8] + roMeans_mono[9] + roMeans_mono[10] + roMeans_mono[11] + roMeans_mono[12] + roMeans_mono[13])/8

roDiff_mono = roMPPAvg_mono - roMAPAvg_mono

siMeans_mono = np.load("commonResults/siMeans_mono.npy")

siMPPAvg_mono = (siMeans_mono[0] + siMeans_mono[1] + siMeans_mono[2] + siMeans_mono[3] + siMeans_mono[4] + siMeans_mono[5])/6
siMAPAvg_mono = (siMeans_mono[6] + siMeans_mono[7] + siMeans_mono[8] + siMeans_mono[9] + siMeans_mono[10] + siMeans_mono[11] + siMeans_mono[12] + siMeans_mono[13])/8

siDiff_mono = siMPPAvg_mono - siMAPAvg_mono

neMeans_mono = np.load("commonResults/neMeans_mono.npy")

neMPPAvg_mono = (neMeans_mono[0] + neMeans_mono[1] + neMeans_mono[2] + neMeans_mono[3] + neMeans_mono[4] + neMeans_mono[5])/6
neMAPAvg_mono = (neMeans_mono[6] + neMeans_mono[7] + neMeans_mono[8] + neMeans_mono[9] + neMeans_mono[10] + neMeans_mono[11] + neMeans_mono[12] + neMeans_mono[13])/8

neDiff_mono = neMPPAvg_mono - neMAPAvg_mono

In [ ]:
etMeans_siamese = np.load("commonResults/etMeans_siamese.npy")

etMPPAvg_siamese = (etMeans_siamese[0] + etMeans_siamese[1] + etMeans_siamese[2] + etMeans_siamese[3] + etMeans_siamese[4] + etMeans_siamese[5])/6
etMAPAvg_siamese = (etMeans_siamese[6] + etMeans_siamese[7] + etMeans_siamese[8] + etMeans_siamese[9] + etMeans_siamese[10] + etMeans_siamese[11] + etMeans_siamese[12] + etMeans_siamese[13])/8

etDiff_siamese = etMPPAvg_siamese - etMAPAvg_siamese

ruMeans_siamese = np.load("commonResults/ruMeans_siamese.npy")

ruMPPAvg_siamese = (ruMeans_siamese[0] + ruMeans_siamese[1] + ruMeans_siamese[2] + ruMeans_siamese[3] + ruMeans_siamese[4] + ruMeans_siamese[5])/6
ruMAPAvg_siamese = (ruMeans_siamese[6] + ruMeans_siamese[7] + ruMeans_siamese[8] + ruMeans_siamese[9] + ruMeans_siamese[10] + ruMeans_siamese[11] + ruMeans_siamese[12] + ruMeans_siamese[13])/8

ruDiff_siamese = ruMPPAvg_siamese - ruMAPAvg_siamese

roMeans_siamese = np.load("commonResults/roMeans_siamese.npy")

roMPPAvg_siamese = (roMeans_siamese[0] + roMeans_siamese[1] + roMeans_siamese[2] + roMeans_siamese[3] + roMeans_siamese[4] + roMeans_siamese[5])/6
roMAPAvg_siamese = (roMeans_siamese[6] + roMeans_siamese[7] + roMeans_siamese[8] + roMeans_siamese[9] + roMeans_siamese[10] + roMeans_siamese[11] + roMeans_siamese[12] + roMeans_siamese[13])/8

roDiff_siamese = roMPPAvg_siamese - roMAPAvg_siamese

siMeans_siamese = np.load("commonResults/siMeans_siamese.npy")

siMPPAvg_siamese = (siMeans_siamese[0] + siMeans_siamese[1] + siMeans_siamese[2] + siMeans_siamese[3] + siMeans_siamese[4] + siMeans_siamese[5])/6
siMAPAvg_siamese = (siMeans_siamese[6] + siMeans_siamese[7] + siMeans_siamese[8] + siMeans_siamese[9] + siMeans_siamese[10] + siMeans_siamese[11] + siMeans_siamese[12] + siMeans_siamese[13])/8

siDiff_siamese = siMPPAvg_siamese - siMAPAvg_siamese

neMeans_siamese = np.load("commonResults/neMeans_siamese.npy")

neMPPAvg_siamese = (neMeans_siamese[0] + neMeans_siamese[1] + neMeans_siamese[2] + neMeans_siamese[3] + neMeans_siamese[4] + neMeans_siamese[5])/6
neMAPAvg_siamese = (neMeans_siamese[6] + neMeans_siamese[7] + neMeans_siamese[8] + neMeans_siamese[9] + neMeans_siamese[10] + neMeans_siamese[11] + neMeans_siamese[12] + neMeans_siamese[13])/8

neDiff_siamese = neMPPAvg_siamese - neMAPAvg_siamese

In [ ]:
etMeans_multi = np.load("commonResults/etMeans_multi.npy")

etMPPAvg_multi = (etMeans_multi[0] + etMeans_multi[1] + etMeans_multi[2] + etMeans_multi[3] + etMeans_multi[4] + etMeans_multi[5])/6
etMAPAvg_multi = (etMeans_multi[6] + etMeans_multi[7] + etMeans_multi[8] + etMeans_multi[9] + etMeans_multi[10] + etMeans_multi[11] + etMeans_multi[12] + etMeans_multi[13])/8

etDiff_multi = etMPPAvg_multi - etMAPAvg_multi

ruMeans_multi = np.load("commonResults/ruMeans_multi.npy")

ruMPPAvg_multi = (ruMeans_multi[0] + ruMeans_multi[1] + ruMeans_multi[2] + ruMeans_multi[3] + ruMeans_multi[4] + ruMeans_multi[5])/6
ruMAPAvg_multi = (ruMeans_multi[6] + ruMeans_multi[7] + ruMeans_multi[8] + ruMeans_multi[9] + ruMeans_multi[10] + ruMeans_multi[11] + ruMeans_multi[12] + ruMeans_multi[13])/8

ruDiff_multi = ruMPPAvg_multi - ruMAPAvg_multi

roMeans_multi = np.load("commonResults/roMeans_multi.npy")

roMPPAvg_multi = (roMeans_multi[0] + roMeans_multi[1] + roMeans_multi[2] + roMeans_multi[3] + roMeans_multi[4] + roMeans_multi[5])/6
roMAPAvg_multi = (roMeans_multi[6] + roMeans_multi[7] + roMeans_multi[8] + roMeans_multi[9] + roMeans_multi[10] + roMeans_multi[11] + roMeans_multi[12] + roMeans_multi[13])/8

roDiff_multi = roMPPAvg_multi - roMAPAvg_multi

siMeans_multi = np.load("commonResults/siMeans_multi.npy")

siMPPAvg_multi = (siMeans_multi[0] + siMeans_multi[1] + siMeans_multi[2] + siMeans_multi[3] + siMeans_multi[4] + siMeans_multi[5])/6
siMAPAvg_multi = (siMeans_multi[6] + siMeans_multi[7] + siMeans_multi[8] + siMeans_multi[9] + siMeans_multi[10] + siMeans_multi[11] + siMeans_multi[12] + siMeans_multi[13])/8

siDiff_multi = siMPPAvg_multi - siMAPAvg_multi

neMeans_multi = np.load("commonResults/neMeans_multi.npy")

neMPPAvg_multi = (neMeans_multi[0] + neMeans_multi[1] + neMeans_multi[2] + neMeans_multi[3] + neMeans_multi[4] + neMeans_multi[5])/6
neMAPAvg_multi = (neMeans_multi[6] + neMeans_multi[7] + neMeans_multi[8] + neMeans_multi[9] + neMeans_multi[10] + neMeans_multi[11] + neMeans_multi[12] + neMeans_multi[13])/8

neDiff_multi = neMPPAvg_multi - neMAPAvg_multi

In [ ]:
etMeans_unsupervised = np.load("commonResults/etMeans_unsupervised.npy")

etMPPAvg_unsupervised = (etMeans_unsupervised[0] + etMeans_unsupervised[1] + etMeans_unsupervised[2] + etMeans_unsupervised[3] + etMeans_unsupervised[4] + etMeans_unsupervised[5])/6
etMAPAvg_unsupervised = (etMeans_unsupervised[6] + etMeans_unsupervised[7] + etMeans_unsupervised[8] + etMeans_unsupervised[9] + etMeans_unsupervised[10] + etMeans_unsupervised[11] + etMeans_unsupervised[12] + etMeans_unsupervised[13])/8

etDiff_unsupervised = etMPPAvg_unsupervised - etMAPAvg_unsupervised

ruMeans_unsupervised = np.load("commonResults/ruMeans_unsupervised.npy")

ruMPPAvg_unsupervised = (ruMeans_unsupervised[0] + ruMeans_unsupervised[1] + ruMeans_unsupervised[2] + ruMeans_unsupervised[3] + ruMeans_unsupervised[4] + ruMeans_unsupervised[5])/6
ruMAPAvg_unsupervised = (ruMeans_unsupervised[6] + ruMeans_unsupervised[7] + ruMeans_unsupervised[8] + ruMeans_unsupervised[9] + ruMeans_unsupervised[10] + ruMeans_unsupervised[11] + ruMeans_unsupervised[12] + ruMeans_unsupervised[13])/8

ruDiff_unsupervised = ruMPPAvg_unsupervised - ruMAPAvg_unsupervised

roMeans_unsupervised = np.load("commonResults/roMeans_unsupervised.npy")

roMPPAvg_unsupervised = (roMeans_unsupervised[0] + roMeans_unsupervised[1] + roMeans_unsupervised[2] + roMeans_unsupervised[3] + roMeans_unsupervised[4] + roMeans_unsupervised[5])/6
roMAPAvg_unsupervised = (roMeans_unsupervised[6] + roMeans_unsupervised[7] + roMeans_unsupervised[8] + roMeans_unsupervised[9] + roMeans_unsupervised[10] + roMeans_unsupervised[11] + roMeans_unsupervised[12] + roMeans_unsupervised[13])/8

roDiff_unsupervised = roMPPAvg_unsupervised - roMAPAvg_unsupervised

siMeans_unsupervised = np.load("commonResults/siMeans_unsupervised.npy")

siMPPAvg_unsupervised = (siMeans_unsupervised[0] + siMeans_unsupervised[1] + siMeans_unsupervised[2] + siMeans_unsupervised[3] + siMeans_unsupervised[4] + siMeans_unsupervised[5])/6
siMAPAvg_unsupervised = (siMeans_unsupervised[6] + siMeans_unsupervised[7] + siMeans_unsupervised[8] + siMeans_unsupervised[9] + siMeans_unsupervised[10] + siMeans_unsupervised[11] + siMeans_unsupervised[12] + siMeans_unsupervised[13])/8

siDiff_unsupervised = siMPPAvg_unsupervised - siMAPAvg_unsupervised

neMeans_unsupervised = np.load("commonResults/neMeans_unsupervised.npy")

neMPPAvg_unsupervised = (neMeans_unsupervised[0] + neMeans_unsupervised[1] + neMeans_unsupervised[2] + neMeans_unsupervised[3] + neMeans_unsupervised[4] + neMeans_unsupervised[5])/6
neMAPAvg_unsupervised = (neMeans_unsupervised[6] + neMeans_unsupervised[7] + neMeans_unsupervised[8] + neMeans_unsupervised[9] + neMeans_unsupervised[10] + neMeans_unsupervised[11] + neMeans_unsupervised[12] + neMeans_unsupervised[13])/8

neDiff_unsupervised = neMPPAvg_unsupervised - neMAPAvg_unsupervised

In [ ]:
etRanking = [etDiff_mono, etDiff_siamese, etDiff_multi, etDiff_unsupervised]
ruRanking = [ruDiff_mono, ruDiff_siamese, ruDiff_multi, ruDiff_unsupervised]
roRanking = [roDiff_mono, roDiff_siamese, roDiff_multi, roDiff_unsupervised]
siRanking = [siDiff_mono, siDiff_siamese, siDiff_multi, siDiff_unsupervised]
neRanking = [neDiff_mono, neDiff_siamese, neDiff_multi, neDiff_unsupervised]

rows = ["MonoTransQuest", "SiameseTransQuest", "MultiTransQuest", "Unsupervised"] ## Since only 4 implementations are present with the code.

plt.figure(figsize=(9, 5))
plt.bar(rows, etRanking)
plt.suptitle('Et-En Ranked Systems')
plt.show()
plt.figure(figsize=(9, 5))
plt.bar(rows, ruRanking)
plt.suptitle('Ru-En Ranked Systems')
plt.show()
plt.figure(figsize=(9, 5))
plt.bar(rows, siRanking)
plt.suptitle('Si-En Ranked Systems')
plt.show()
plt.figure(figsize=(9, 5))
plt.bar(rows, roRanking)
plt.suptitle('Ro-En Ranked Systems')
plt.show()
plt.figure(figsize=(9, 5))
plt.bar(rows, neRanking)
plt.suptitle('Ne-En Ranked Systems')
plt.show()